## Create a project and import required modules

In [1]:
import os
import requests
import time
from xml.etree import ElementTree

## Set the subscription key and create a prompt for TTS

In [2]:
try:
    input = raw_input
except NameError:
    pass

## Let's create a class. This is where we'll put our methods for token exchange, and calling the text-to-speech API.

In [3]:
class TextToSpeech(object):
    def __init__(self, subscription_key):
        self.subscription_key = subscription_key
        self.tts = input("What would you like to convert to speech: ")
        self.timestr = time.strftime("%Y%m%d-%H%M")
        self.access_token = None
    
# Get an access token
    
    def get_token(self):
        fetch_token_url = "https://francecentral.api.cognitive.microsoft.com/sts/v1.0/issuetoken"
        headers = {
            'Ocp-Apim-Subscription-Key': self.subscription_key
        }
        response = requests.post(fetch_token_url, headers=headers)
        self.access_token = str(response.text)
        
# Make a request and save the response       

    def save_audio(self):
        base_url = 'https://francecentral.tts.speech.microsoft.com/'
        path = 'cognitiveservices/v1'
        constructed_url = base_url + path
        headers = {
            'Authorization': 'Bearer ' + self.access_token,
            'Content-Type': 'application/ssml+xml',
            'X-Microsoft-OutputFormat': 'riff-24khz-16bit-mono-pcm',
            'User-Agent': 'Pierre-Speech'
        }
        xml_body = ElementTree.Element('speak', version='1.0')
        xml_body.set('{http://www.w3.org/XML/1998/namespace}lang', 'fr-FR')
        voice = ElementTree.SubElement(xml_body, 'voice')
        voice.set('{http://www.w3.org/XML/1998/namespace}lang', 'fr-FR')
        voice.set(
            'name', 'Microsoft Server Speech Text to Speech Voice (fr-FR, Paul, Apollo)')
        voice.text = self.tts
        body = ElementTree.tostring(xml_body)

        response = requests.post(constructed_url, headers=headers, data=body)
        if response.status_code == 200:
            with open('sample-' + self.timestr + '.wav', 'wb') as audio:
                audio.write(response.content)
                print("\nStatus code: " + str(response.status_code) +
                      "\nYour TTS is ready for playback.\n")
        else:
            print("\nStatus code: " + str(response.status_code) +
                  "\nSomething went wrong. Check your subscription key and headers.\n")

## Put it all together

In [4]:
if __name__ == "__main__":
    subscription_key = "5768554d391947a588a7f9a5826f21a5"
    app = TextToSpeech(subscription_key)
    app.get_token()
    app.save_audio()


What would you like to convert to speech: Bonjour j'aime la baguette

Status code: 200
Your TTS is ready for playback.

